# Student Loan Risk with Deep Learning

In [7]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [8]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [9]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [10]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [12]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"].values

# Display a sample of y
print(y[:5])

[0 0 0 1 0]


In [14]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop("credit_ranking", axis=1).values

# Review the features DataFrame
print(X)

[[ 7.4    0.7    0.    ...  3.51   0.56   9.4  ]
 [ 7.8    0.88   0.    ...  3.2    0.68   9.8  ]
 [ 7.8    0.76   0.04  ...  3.26   0.65   9.8  ]
 ...
 [ 6.3    0.51   0.13  ...  3.42   0.75  11.   ]
 [ 5.9    0.645  0.12  ...  3.57   0.71  10.2  ]
 [ 6.     0.31   0.47  ...  3.39   0.66  11.   ]]


### Step 3: Split the features and target sets into training and testing datasets.


In [15]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [16]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_features = X_scaler.transform(X_train)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [26]:
# Define the the number of inputs (features) to the model
eleven_features = len(X_train[0])

# Review the number of features
print(eleven_features)

11


In [72]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 11

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 5

# Define the number of neurons in the output layer
number_output_neurons = 1

In [68]:
# Create the Sequential model instance
nn = Sequential()

# Add the first hidden layer
nn.add(Dense(units=hidden_nodes_layer1, input_dim=eleven_features, activation="relu"))

# Add the second hidden layer
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=number_output_neurons, activation="sigmoid"))


In [69]:
# Display the Sequential model summary
nn.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                │ (None, 11)             │           132 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 6)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │             7 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 211 (844.00 B)

 Trainable params: 211 (844.00 B)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [70]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [73]:
# Fit the model using 50 epochs and the training data
fit_model = nn.fit(X_train, y_train, epochs=50, batch_size=32)

Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 766us/step - accuracy: 0.7350 - loss: 0.5346
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 746us/step - accuracy: 0.7031 - loss: 0.5578
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 806us/step - accuracy: 0.7360 - loss: 0.5326
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 802us/step - accuracy: 0.7203 - loss: 0.5412
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 661us/step - accuracy: 0.7049 - loss: 0.5540
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 713us/step - accuracy: 0.7294 - loss: 0.5419
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 750us/step - accuracy: 0.7241 - loss: 0.5615
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 821us/step - accuracy: 0.7187 - loss: 0.5512
Epoch 9/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 710us/step - accuracy: 0.7171 - loss: 0.5537
Epoch 10/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 645us/step - accuracy: 0.7238 - loss: 0.5551
Epoch 11/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 802us/step - accuracy: 0.7293 - loss: 0.5359
Epoch 12/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 757us/step

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [74]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
nn_model_loss, nn_model_accuracy = nn.evaluate(X_test, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {nn_model_loss}, Accuracy: {nn_model_accuracy}")

13/13 - 0s - 8ms/step - accuracy: 0.7325 - loss: 0.5400
Loss: 0.5400171875953674, Accuracy: 0.7325000166893005


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [75]:
# Set the model's file path
file_path = Path("student_loan_model.keras")

# Export your model to a keras file
nn.save(file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [76]:
# Set the model's file path
#file path is already defined above

# Load the model to a new object
nn_model_loaded=tf.keras.models.load_model(file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [78]:
# Make predictions with the test data
nn_model_predictions = nn_model_loaded.predict(X_test)

# Display a sample of the predictions
nn_model_predictions[:5]

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


array([[0.7493291 ],
       [0.28146333],
       [0.88786006],
       [0.7366927 ],
       [0.9688221 ]], dtype=float32)

In [79]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame(nn_model_predictions, columns=["predictions"])
predictions_df["predictions"] = round(predictions_df["predictions"], 0)
predictions_df.head()

,predictions
0,1.0
1,0.0
2,1.0
3,1.0
4,1.0


### Step 4: Display a classification report with the y test data and predictions

In [80]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df["predictions"]))

              precision    recall  f1-score   support

           0       0.73      0.68      0.70       188
           1       0.73      0.78      0.76       212

    accuracy                           0.73       400
   macro avg       0.73      0.73      0.73       400
weighted avg       0.73      0.73      0.73       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate. 
The data that I would need to collect to build a recommendation system would be 3 fold.  First I would collect the student's demographics like age, geographic location, Field of study, type of school, & GPA.  Second, Financial history like credit score, employment status, income, existing debt, savings, any other financial aide, or Family income. Thirdly, Loan Details like Amount needed, Interest rate and terms.  All this information would relevant as it will help us understand the student's specific educational and career goals. Their educational goals and career choices would provide insight to the student's financial literacy and expected earnings after graduation.  The Load details enables matching students with loans they're eligible for and that best fit their needs.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.  
I would combine Content-Based filtering and Context-based filtering.  Content-based to make sure details like loan preferences and terms can be properly matched with specific educational requirements and career endeavors. The context-based filtering like age, field of study and geographic location provide insight to population trends and indicators that can help align students historically successful student loans.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.  Two real-world challenges to consider when building this recommendation system:

One significant real-world challenge is ensuring the privacy and security of the data collected. This is particularly difficult because sensitive data is essential for making accurate recommendations, yet collecting and storing such information increases the risk of privacy breaches. Additional resources must be allocated to secure and maintain this sensitive data.
Another critical challenge is addressing the ethical considerations related to potential biases. There is a risk that the recommendation system could perpetuate or exacerbate existing inequalities in access to educational financing. Ensuring fairness and avoiding discriminatory outcomes is crucial but challenging, as biases can be subtle and deeply embedded in historical data or seemingly neutral factors.
These challenges are especially concerning because student loans have long-term financial implications. Any biases or privacy breaches could have severe consequences for the affected individuals.
